### Algoritmo NSGAII

O Algoritmo NSGAII trabalha como um operador de seleção de indivíduos sobre uma população. É mais rápido e possui uma menor complexidade computacional se comparado com outros algoritmos notórios. A adoção do método elitista garante uma maior velocidade de sua execução.

Inicialmente uma população P(t) de tamanho N (t=0) é criada aleatoriamente e ordenada de acordo com sua não-dominância. Para cada solução é atribuído um valor de fitness (ou rank) equivalente ao seu nível de não-dominância, sendo o nível 1 o melhor, nível 2 o melhor seguinte e etc.

Em seguida Q(t), de tamanho N e t=0, é gerado utilizando operadores de seleção de torneiro binário, recombinação e mutação.

Feito isso, combina-se P(t) e Q(t) (pais e filhos) obtendo-se R(t) de tamanho 2N. O elitismo é garantido neste processo pois, até o momento, todos os indivíduos gerados estão preservados. 

As soluções de R(t) são ordenadas de acordo com sua não-dominância, formando as fronteiras, desde F1 (fronteira que contém as melhores soluções) até a última identificada. O pior caso ocorre quando para cada fronteira há apenas uma solução.

O objetivo seguinte é obter a próxima geração P(t+1) de tamanho N a partir de R(t). Logo, se o tamanho da fronteira F1 de R(t) for menor que N todas as soluções de F1 passarão a pertencer a P(t+1). E soluções das fronteiras seguintes são adicionadas a P(t+1) até que seu tamanho seja N, de acordo com o ranking de não dominância.

<div align='center'><img src='https://www.researchgate.net/profile/N_Nariman-Zadeh/publication/228569970/figure/fig1/AS:302014627106822@1449017306661/Fig-1-Basics-of-NSGA-II-procedure.png' width='56%' align='center'/>
Fonte: https://www.researchgate.net/publication/228569970_Thermodynamic_Pareto_optimization_of_turbojet_engines_using_multi-objective_genetic_algorithms</div>

Durante a geração de P(t+1) calcula-se para cada indivíduo seu ranking e distância de agrupamento, necessários na hora de comparar as soluções e obter a melhor através do operador crowded-comparison.

Agora, sobre a nova população P(t+1) de tamanho N utilizamos operadores de seleção, mutação e cruzamento para criar a nova população Q(t+1) de tamanho N. 

Este processo se repete até que um critério de parada seja atingido. A função NSGA2 é composta por outras funções auxiliares, conforme descrito a seguir.


In [1]:
#função responsável por determinar a dominância de um indivíduo sobre o outro.
#ind1 dominará ind2 se:
#    - ind1 e ind2 forem diferentes entre si e 
#    - ind1 preceder ind2
def dominates(ind1, ind2):
    
    #verifica igualdade
    #r1 = np.array_equal(ind1, ind2)
    #if r1 == True:
    #    return False
    
    #verifica a precedência  
    #for i in range(len(ind1)):               
    #    if ind1[i] > ind2[i]:
    #        return False       
        
    #return True
    
    'Returns `True` if `ind1` dominates `ind2` - assumes min. problem.'
    extrictly_better = False
    
    for obj1 in ind1.fitness.values:        
        for obj2 in ind2.fitness.values:
            
            if obj1 > obj2:                
                return False
            if not extrictly_better and obj1 < obj2:
                extrictly_better = True
                
    return extrictly_better    

In [2]:
#função que identifica as soluções de acordo com cada fronteira existente, classificando-os segundo
#    seu nível de não dominância
def fast_non_dominated_sort(P):
    
    F = {} #lista(dicionário) das fronteiras
   
    #início do loop que analisa cada indivíduo p da população P
    #este primeiro laço é responsável por identificar os indivíduos da primeira fronteira
    for p in P:
    
        
        n_p = 0 #contador de dominância, quantidade de soluções que p domina, todas as soluções da primeira
            #fronteira possuem n_p = 0
        S_p = [] #lista das soluções dominadas por p
        
        #compara a solução p com as demais soluções da população P, determinando sua relação de dominância
        for q in P:
            
            if dominates(p, q): #se p domina q, q é adicionado ao conjunto de soluções dominadas por p
                S_p.append(q) 
                #p.dominated_solutions.append(q)
            elif dominates(q, p): #senão o contador de dominância de p é incrementado
                n_p = n_p + 1
            #print('p= ', p, 'q= ', q, '\n', dominates(p, q), dominates(q, p))
          
        #se o contador de dominância do p em questão for igual a zero ele pertencerá à primeira fronteira
        if n_p == 0:
            p.rank = 1 #soluções da fronteira 1 possuem rank igual a 1           
            if 1 in F:
                F[1].append(p) 
                #print('>>> ', F[1])
            else:
                F[1] = [p] 
                #print('### ', F[1])
                #print('###\n ')
        
        p.dominated_solutions = S_p
        p.domination_counter = n_p
        
        #print('sp=', S_p, 'np=', n_p, '############################\n ')
        
    #print('&&& ', F[1])
    #este segundo laço é responsável por identificar os indivíduos das demais fronteiras
    i = 1
    for x in F[i]:
        Q = [] #armazena os indivíduos da próxima fronteira     
        
        for p in F[i]:
            #for q in S_p:
            for q in p.dominated_solutions:
                #n_q = p.domination_counter - 1
                q.domination_counter = q.domination_counter - 1
                
                if q.domination_counter == 0:
                    q.rank = i + 1
                    Q.append(q)
                    
                #print('n_q= ', q.domination_counter, 'p= ', p, 'q= ', q, '\n\n', 'pds=', p.dominated_solutions, 'pdc', p.domination_counter)
                    
        i = i + 1
        F[i] = Q         
        
    return F

In [3]:
def crowding_distance_assignmnetBKP(I):
    
    l = len(I)
    individuals = []

    for i in I:
        individuals.append((i,0));
        
        individuals.sort()

    return individuals

In [4]:
def crowding_distance_assignmnet(I):
    
        if len(l) > 0:
            solutions_num = len(l)
            for i in l:
                i.crowding_distance = 0
            
            for m in range(len(l[0].objectives)):
                l = sorted(l, cmp=functools.partial(self.__sort_objective, m=m))
                l[0].crowding_distance = self.problem.max_objectives[m]
                l[solutions_num-1].crowding_distance = self.problem.max_objectives[m]
                for index, value in enumerate(front[1:solutions_num-1]):
                    l[index].crowding_distance = (front[index+1].crowding_distance - front[index-1].crowding_distance) / (self.problem.max_objectives[m] - self.problem.min_objectives[m])

In [5]:
def crowded_comparison_operator(ind1, ind2):
    
    if (ind1.rank < ind2.rank) or ((ind1.rank == ind2.rank) and (ind1.crowding_distance > ind2.crowding_distance)):
        return True
    else:
        return False

In [6]:
def rodrigoNSGA2(pop,tam):
    
    #ordeno os indivíduos da população de acordo com sua não-dominância
    F = fast_non_dominated_sort(pop)
    
    i=1
    P = {}
    P[t+1] = 0
    while len(P[t+1])+len(F[i]) <= tam:
        crowding_distance_assignmnet(F[i])
        P[t+1] = P[t+1].append(F[i])
        i = i + 1
        
    #classifica as soluções em ordem decrescente usando o operador crowded_comparison
    crowded_comparison_operator(F[i])
    
    P[t+1] = P[t+1].append(F[i][1:(tam - len(P[t+1]))])
    Q[t+1] = make_new_pop(P[t+1])
    
    t = t + 1    
    

In [7]:
import time, array, random, copy, math
import numpy as np
import pandas as pd

from pprint import pprint

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rcParams['text.latex.preamble'] ='\\usepackage{libertine}\n\\usepackage[utf8]{inputenc}'

import seaborn
seaborn.set(style='whitegrid')
seaborn.set_context('notebook')

In [9]:
from deap import algorithms, base, benchmarks, tools, creator

In [10]:
random.seed(a=42)

In [11]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0))
#creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMin)
#creator.create("Individual", list, typecode='d', fitness=creator.FitnessMin)
#creator.create("Individual", array.array, 
creator.create("Individual", list, 
                #typecode='d', 
                fitness=creator.FitnessMin,
                rank = None,
                crowding_distance = None,
                #dominated_solutions = set(),
                dominated_solutions = [],
                features = None,
                objectives = None,
                #dominates = None,
                domination_counter = 0,
              )

In [12]:
def uniform(low, up, size=None):
    try:
        return [random.uniform(a, b) for a, b in zip(low, up)]
    except TypeError:
        return [random.uniform(a, b) for a, b in zip([low] * size, [up] * size)]

In [13]:
toolbox = base.Toolbox()

In [54]:
NDIM = 30
BOUND_LOW, BOUND_UP = 0.0, 1.0
toolbox.register("evaluate", lambda ind: benchmarks.dtlz3(ind, 2))

In [55]:
toolbox.register("attr_float", uniform, BOUND_LOW, BOUND_UP, NDIM)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_float)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0)
toolbox.register("mutate", tools.mutPolynomialBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0, indpb=1.0/NDIM)
#toolbox.register("select", tools.selNSGA2)
toolbox.register("select", rodrigoNSGA2)

In [56]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
a=toolbox.population(10)

In [57]:
a[0].fitness

deap.creator.FitnessMin(())

In [63]:
for i in a:
    i.fitness = toolbox.evaluate(i)
    print(i.fitness, '\n')

[2223.146491006109, 1720.1180376947552] 

[865.9164559808287, 2930.678764755617] 

[453.76798268541233, 2984.0445501527684] 

[932.3883247878811, 3043.5174883777013] 

[1082.955691434291, 2383.9393439175233] 

[466.8329718745695, 3033.9426704262187] 

[2520.83194560357, 1920.3378413670891] 

[508.9993321381952, 3124.9362678730204] 

[3189.4706560034597, 2170.353776012462] 

[3585.010951682069, 1237.8785236747037] 



In [59]:
a[0].fitness

[2223.146491006109, 1720.1180376947552]

In [18]:
f=fast_non_dominated_sort(a)

In [35]:
#f

In [20]:
len(f)

51

In [21]:
len(f[1])

50

In [22]:
f[1]

[[0.6394267984578837, 0.025010755222666936],
 [0.27502931836911926, 0.22321073814882275],
 [0.7364712141640124, 0.6766994874229113],
 [0.8921795677048454, 0.08693883262941615],
 [0.4219218196852704, 0.029797219438070344],
 [0.21863797480360336, 0.5053552881033624],
 [0.026535969683863625, 0.1988376506866485],
 [0.6498844377795232, 0.5449414806032167],
 [0.2204406220406967, 0.5892656838759087],
 [0.8094304566778266, 0.006498759678061017],
 [0.8058192518328079, 0.6981393949882269],
 [0.3402505165179919, 0.15547949981178155],
 [0.9572130722067812, 0.33659454511262676],
 [0.09274584338014791, 0.09671637683346401],
 [0.8474943663474598, 0.6037260313668911],
 [0.8071282732743802, 0.7297317866938179],
 [0.5362280914547007, 0.9731157639793706],
 [0.3785343772083535, 0.552040631273227],
 [0.8294046642529949, 0.6185197523642461],
 [0.8617069003107772, 0.577352145256762],
 [0.7045718362149235, 0.045824383655662215],
 [0.22789827565154686, 0.28938796360210717],
 [0.0797919769236275, 0.232790886361

In [23]:
a[1]

[0.27502931836911926, 0.22321073814882275]

In [24]:
a[2]

[0.7364712141640124, 0.6766994874229113]

In [25]:
a[3]

[0.8921795677048454, 0.08693883262941615]

In [26]:
dominates(a[1],a[2])

False

In [27]:
dominates(a[2],a[3])

False

In [28]:
toolbox.pop_size = 50
toolbox.max_gen = 1000
toolbox.mut_prob = 0.2

In [29]:
#p = toolbox.population(n=toolbox.pop_size)

In [30]:
#pprint(vars(p[0]))

In [31]:
#toolbox.select(p[0], len(p[0]))

In [32]:
def run_ea(toolbox, stats=None, verbose=False):
    pop = toolbox.population(n=toolbox.pop_size)
    pop = toolbox.select(pop, len(pop))   
    return algorithms.eaMuPlusLambda(pop, 
                                     toolbox, 
                                     mu=toolbox.pop_size, 
                                     lambda_=toolbox.pop_size, 
                                     cxpb=1-toolbox.mut_prob,
                                     mutpb=toolbox.mut_prob, 
                                     stats=stats, 
                                     ngen=toolbox.max_gen, 
                                     verbose=verbose)

In [33]:
run_ea(toolbox)

UnboundLocalError: local variable 't' referenced before assignment

In [ ]:
%time res,_ = run_ea(toolbox)

In [ ]:
fronts = tools.emo.sortLogNondominated(res, len(res))

In [ ]:
plot_colors = seaborn.color_palette("Set1", n_colors=10)
fig, ax = plt.subplots(1, figsize=(4,4))
for i,inds in enumerate(fronts):
    par = [toolbox.evaluate(ind) for ind in inds]
    df = pd.DataFrame(par)
    df.plot(ax=ax, kind='scatter', label='Front ' + str(i+1), 
                 x=df.columns[0], y=df.columns[1], 
                 color=plot_colors[i])
plt.xlabel('$f_1(\mathbf{x})$');plt.ylabel('$f_2(\mathbf{x})$');